In [ ]:
%pylab inline
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import scipy

## E1z2

In [ ]:
metadata = pd.read_csv('data/mousedata_2020/E1z2/metadata.csv',index_col=0)
metadata

In [ ]:
expdata = pd.read_csv('data/mousedata_2020/E1z2/simu_sc_counts.csv',index_col=0)
expdata = expdata.T

In [ ]:
expdata

In [ ]:
min_x = metadata.x_global.min()
min_y = metadata.y_global.min()
max_x = metadata.x_global.max()
max_y = metadata.y_global.max()
dis_x = max_x - min_x
dis_y = max_y - min_y

In [ ]:
xv, yv = np.meshgrid(np.linspace(min_x, max_x, 60),np.linspace(min_y, max_y, 60))

In [ ]:
xbin = np.diff(xv[0])[0]
ybin = np.diff(yv[:,0])[0]
xbin,ybin

In [ ]:
figsize(5,7)
scatter(metadata.x_global,metadata.y_global,s=0.5)
scatter(xv.flatten(),yv.flatten(),s=5)


In [ ]:
indicator = np.zeros_like(xv)

In [ ]:
xy_coord = metadata[['x_global','y_global']].values
xy_coord

In [ ]:
annotationdf =  pd.get_dummies(metadata.celltype_mapped_refined)

In [ ]:
annotationdf

In [ ]:
from tqdm import tqdm
indicator = np.zeros(xv.shape[0]*xv.shape[1])
st_counts = np.zeros([xv.shape[0]*xv.shape[1],expdata.shape[1]])
st_anno = np.zeros([xv.shape[0]*xv.shape[1],annotationdf.shape[1]])
isselected=[]
for i in tqdm(range(xv.shape[0])):
    for j in range(xv.shape[1]):
        xbmin = xv[i][j]-xbin/100*70/2
        xbmax = xv[i][j]+xbin/100*70/2
        ybmin = yv[i][j]-ybin/100*70/2
        ybmax = yv[i][j]+ybin/100*70/2
        tmp=[]
        for x in range(xy_coord.shape[0]):
            if xy_coord[x][0]>xbmin and xy_coord[x][0]<=xbmax and xy_coord[x][1]>ybmin and xy_coord[x][1]<=ybmax:
                st_counts[i*xv.shape[1]+j]+=expdata.values[x]
                indicator[i*xv.shape[1]+j]+=1
                st_anno[i*xv.shape[1]+j]+=annotationdf.values[x]
                tmp.append(x)
        isselected.append(tmp)

In [ ]:
figsize(6,8)
scatter(metadata.x_global,metadata.y_global,s=0.5)
scatter(xv.flatten(),yv.flatten(),c=(indicator>=2),s=10,cmap='cool')

In [ ]:
figsize(3,3)
hist(indicator,bins=40)

In [ ]:
stdata_df = pd.DataFrame(st_counts,columns=expdata.columns,index=['ST_'+str(i) for i in range(st_counts.shape[0])])

stmetadata = pd.DataFrame([xv.flatten(),yv.flatten()],columns=stdata_df.index,index=['coord_x','coord_y'])
stmetadata = stmetadata.T

stannodf = pd.DataFrame(st_anno,columns=annotationdf.columns)
stannodf.index = stmetadata.index

In [ ]:
stdata_df = stdata_df[indicator>=3]
stmetadata = stmetadata[indicator>=3]
stannodf = stannodf[indicator>=3]
stannodf = stannodf.div(stannodf.sum(1),0)

In [ ]:
stdata_df

In [ ]:
indicator.shape

In [ ]:
mask = (indicator >= 3)                      # boolean mask
isslectednp = np.array(isselected, dtype=object)[mask]


In [ ]:
idxslected =[]
for x in isslectednp.tolist():
    idxslected = idxslected+x
idxslected = sorted(list(set(idxslected)))

In [ ]:
metadatac = np.zeros(metadata.shape[0])

In [ ]:
metadatac[idxslected]=1

In [ ]:
figsize(12,8)
subplot(121)
plt.scatter(metadata.x_global,-metadata.y_global,c=metadatac,s=2,cmap='bwr')

In [ ]:
figsize(12,8)
subplot(121)
plt.scatter(metadata.x_global,-metadata.y_global,c=expdata.Cd34,s=2,cmap='hot')
subplot(122)
plt.scatter(stmetadata.coord_x,-stmetadata.coord_y,c=stdata_df.Cd34,s=50,cmap='hot')

In [ ]:
stmetadata.to_csv('data/mousedata_2020/E1z2/simu_st_metadata_et.csv')
stdata_df.T.to_csv('data/mousedata_2020/E1z2/simu_st_counts_et.csv')

In [ ]:
stannodf.to_csv('data/mousedata_2020/E1z2/simu_st_celltype_et.csv')